In [1]:
import math
import logging
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
from packaging import version
from torch.nn import init
from tensorly.tenalg import multi_mode_dot
from transformers.generation_utils import GenerationMixin
from transformers.file_utils import PushToHubMixin
from transformers.modeling_utils import (ModuleUtilsMixin,
                                         apply_chunking_to_forward, 
                                         find_pruneable_heads_and_indices,
                                         prune_linear_layer)
from transformers.models.bart.modeling_bart import BartEncoderLayer, BartDecoderLayer
from transformers.activations import gelu_new
from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions, 
    CausalLMOutputWithCrossAttentions
)


import random
import math
import logging
import argparse
from tqdm import tqdm

import yaml
import hydra
from omegaconf import OmegaConf

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_metric, load_from_disk
from transformers import AutoConfig, AutoTokenizer
from transformers import set_seed, get_cosine_schedule_with_warmup, AdamW

from model import GrafomerModel
from utils import preprocess_function_with_setting, load_data, postprocess_text, CustomDataCollator


from transformers import AutoConfig
from transformers.generation_utils import GenerationMixin
from transformers.file_utils import PushToHubMixin
from transformers.modeling_utils import ModuleUtilsMixin
from transformers.modeling_outputs import BaseModelOutput, Seq2SeqLMOutput
from transformers.models.bart.modeling_bart import BartEncoderLayer, BartDecoderLayer
from transformers.models.bart.modeling_bart import _expand_mask, _make_causal_mask

from typing import Tuple

# Weight Distillation

In [107]:
TeacherWeightGroup.set_network(model)

In [108]:
WeightGenerator('encoder.attention.self.query', 1, 2, 768, 768)()

tensor([[-0.0593,  0.0244,  0.0095,  ..., -0.0239, -0.0274,  0.0233],
        [ 0.1087,  0.0397, -0.0091,  ...,  0.0201, -0.0182, -0.0401],
        [ 0.0575, -0.0086,  0.0265,  ..., -0.0381, -0.0725, -0.0275],
        ...,
        [ 0.0110,  0.0134, -0.0536,  ...,  0.0412, -0.0466,  0.0353],
        [ 0.0271, -0.0443,  0.0359,  ..., -0.0067, -0.0521,  0.0614],
        [-0.0270, -0.0813, -0.0319,  ...,  0.0358, -0.0717, -0.0470]],
       grad_fn=<AddBackward0>)

In [42]:
a = torch.rand(768, 768, 6).transpose(2, 0)

In [43]:
li = []
for _ in range(6):
    a = torch.rand(768, 768)
    li.append(a)
torch.stack(li, dim=-1).size()

torch.Size([768, 768, 6])

In [44]:
a = torch.rand(768, 768, 6)
print(a.size())
l = torch.rand(6, 1)
a.matmul(l).size()

torch.Size([768, 768, 6])


torch.Size([768, 768, 1])

In [119]:
li = []
a = torch.rand(768, 768)
li.append(a)

pooler_out = torch.stack(li, dim=-1)
w_l = nn.Parameter(torch.empty(1, 1))
out = pooler_out.matmul(w_l)


In [121]:
pooler_out.squeeze()

tensor([[0.4825, 0.3369, 0.1921,  ..., 0.7776, 0.3765, 0.8853],
        [0.4086, 0.6695, 0.6508,  ..., 0.5547, 0.7219, 0.8283],
        [0.4493, 0.2959, 0.4563,  ..., 0.7574, 0.4172, 0.0586],
        ...,
        [0.9057, 0.3078, 0.1581,  ..., 0.6746, 0.4088, 0.0066],
        [0.8304, 0.8895, 0.7289,  ..., 0.4575, 0.6875, 0.9318],
        [0.4120, 0.8666, 0.7388,  ..., 0.9489, 0.7431, 0.4463]])

In [124]:
out.squeeze()

tensor([[-7.0701e-29, -4.9374e-29, -2.8144e-29,  ..., -1.1394e-28,
         -5.5171e-29, -1.2973e-28],
        [-5.9882e-29, -9.8099e-29, -9.5361e-29,  ..., -8.1278e-29,
         -1.0579e-28, -1.2138e-28],
        [-6.5844e-29, -4.3360e-29, -6.6857e-29,  ..., -1.1099e-28,
         -6.1142e-29, -8.5914e-30],
        ...,
        [-1.3272e-28, -4.5098e-29, -2.3160e-29,  ..., -9.8856e-29,
         -5.9910e-29, -9.6764e-31],
        [-1.2169e-28, -1.3035e-28, -1.0681e-28,  ..., -6.7034e-29,
         -1.0075e-28, -1.3654e-28],
        [-6.0370e-29, -1.2699e-28, -1.0826e-28,  ..., -1.3904e-28,
         -1.0889e-28, -6.5401e-29]], grad_fn=<SqueezeBackward0>)

In [228]:
StudentLinear('encoder.pooler.dense', 2, 2, 768, 768)

StudentLinear(
  (weight_generator): WeightGenerator(
    (tanh): Tanh()
  )
  (bias_generator): BiasGenerator(
    (tanh): Tanh()
  )
)

In [45]:
TeacherWeightGroup.set_network(model)

In [220]:
WeightGenerator('encoder.attention.self.query', 1, 2, 768, 768)().size()

torch.Size([768, 768])

In [221]:
BiasGenerator('encoder.attention.self.query', 1, 2, 768)().size()

torch.Size([768])

In [210]:
WeightGenerator('encoder.intermediate.dense', 2, 2, 768, 3072)().size()

torch.Size([3072, 768])

In [211]:
BiasGenerator('encoder.intermediate.dense', 2, 2, 3072)().size()

torch.Size([3072])

In [224]:
intermediate = StudentLinear('encoder.intermediate.dense', 2, 2, 768, 3072)

In [225]:
input = torch.rand(1024, 768)
intermediate(input).size()

torch.Size([1024, 3072])

In [226]:
input = intermediate(input)
output = StudentLinear('encoder.output.dense', 2, 2, 3072, 768)
output(input).size()

torch.Size([1024, 768])

In [227]:
for name, _ in model.named_parameters():
    print(name)

encoder.embeddings.word_embeddings.weight
encoder.embeddings.position_embeddings.weight
encoder.embeddings.token_type_embeddings.weight
encoder.embeddings.LayerNorm.weight
encoder.embeddings.LayerNorm.bias
encoder.encoder.layer.0.attention.self.query.weight
encoder.encoder.layer.0.attention.self.query.bias
encoder.encoder.layer.0.attention.self.key.weight
encoder.encoder.layer.0.attention.self.key.bias
encoder.encoder.layer.0.attention.self.value.weight
encoder.encoder.layer.0.attention.self.value.bias
encoder.encoder.layer.0.attention.output.dense.weight
encoder.encoder.layer.0.attention.output.dense.bias
encoder.encoder.layer.0.attention.output.LayerNorm.weight
encoder.encoder.layer.0.attention.output.LayerNorm.bias
encoder.encoder.layer.0.intermediate.dense.weight
encoder.encoder.layer.0.intermediate.dense.bias
encoder.encoder.layer.0.output.dense.weight
encoder.encoder.layer.0.output.dense.bias
encoder.encoder.layer.0.output.LayerNorm.weight
encoder.encoder.layer.0.output.LayerNorm

In [2]:
class TeacherWeightGroup:
    teacher_model: nn.Module = None

    @classmethod
    def set_network(cls, teacher_model: nn.Module):
        TeacherWeightGroup.teacher_model = teacher_model

    @classmethod
    def generate_weight_group(
        cls, 
        weight_class_name: str, 
        current_layer_index: int, 
        num_student_layers: int
    ):
        part, weight_class_name = weight_class_name.split(".", 1)
        weight_class_name += ".weight"
        weight_instances = list()

        if part == "encoder":
            for instance_name, instance in TeacherWeightGroup.teacher_model.encoder.named_parameters():
                if "attention" in weight_class_name:
                    if weight_class_name in instance_name and 'attention' in instance_name:
                        weight_instances.append(instance)
                else:
                    if weight_class_name in instance_name and 'attention' not in instance_name:
                        weight_instances.append(instance)

        elif part == "decoder":
            for instance_name, instance in TeacherWeightGroup.teacher_model.decoder.named_parameters():
                # Conv1d 연산의 파라미터의 in, out이 linear 연산 파라미터의 반대로 되어있음
                # 반복문 돌때마다 transpose하지 않고 끝난 후 조정해주면 더 좋을 것 같은데 이후에 시간있으면 수정..
                instance = instance.T
                if "attention" in weight_class_name:
                    if weight_class_name in instance_name and 'attention' in instance_name:
                        weight_instances.append(instance)
                else:
                    if weight_class_name in instance_name and 'attention' not in instance_name:
                        weight_instances.append(instance)
        
        weight_instances = torch.stack(weight_instances, dim=-1)
        teacher_network_layers = weight_instances.size()[-1]
        
        start = (current_layer_index - 1) * int(teacher_network_layers / num_student_layers)
        end = current_layer_index * int(teacher_network_layers / num_student_layers)
        return weight_instances[:, :, start:end]
    
    @classmethod
    def generate_bias_group(
        cls, 
        weight_class_name: str, 
        current_layer_index: int, 
        num_student_layers: int
    ):
        part, weight_class_name = weight_class_name.split(".", 1)
        weight_class_name += ".bias"
        weight_instances = list()

        if part == "encoder":
            for instance_name, instance in TeacherWeightGroup.teacher_model.encoder.named_parameters():
                if "attention" in weight_class_name:
                    if weight_class_name in instance_name and 'attention' in instance_name:
                        weight_instances.append(instance)
                else:
                    if weight_class_name in instance_name and 'attention' not in instance_name:
                        weight_instances.append(instance)

        elif part == "decoder":
            for instance_name, instance in TeacherWeightGroup.teacher_model.decoder.named_parameters():
                if "attention" in weight_class_name:
                    if weight_class_name in instance_name and 'attention' in instance_name:
                        weight_instances.append(instance)
                else:
                    if weight_class_name in instance_name and 'attention' not in instance_name:
                        weight_instances.append(instance)
        
        weight_instances = torch.stack(weight_instances, dim=-1)
        teacher_network_layers = weight_instances.size()[-1]
        
        start = (current_layer_index - 1) * int(teacher_network_layers / num_student_layers)
        end = current_layer_index * int(teacher_network_layers / num_student_layers)
        return weight_instances[:, start:end]


class WeightGenerator(nn.Module):
    def __init__(
        self,
        weight_class_name: str,
        current_layer_index: int,
        num_student_layers: int,
        student_weight_in: int,
        student_weight_out: int
    ):
        super().__init__()
        self.subset = TeacherWeightGroup.generate_weight_group(
            weight_class_name, current_layer_index, num_student_layers
        )
        num_adjacent_layers, teacher_weight_out, teacher_weight_in = self.subset.shape

        # self.W_i = nn.Parameter(torch.empty(student_weight_in, teacher_weight_in))
        # self.W_o = nn.Parameter(torch.empty(student_weight_out, teacher_weight_out))
        self.W_l = nn.Parameter(torch.empty(1, num_adjacent_layers))
        self.W = nn.Parameter(torch.ones(student_weight_out, student_weight_in))
        self.B = nn.Parameter(torch.zeros(student_weight_out, student_weight_in))

        self.tanh = nn.Tanh()
        self.init_weights_()
    
    def init_weights_(self):
        # init.xavier_uniform_(self.W_i)
        # init.xavier_uniform_(self.W_o)
        init.xavier_uniform_(self.W_l)

    def forward(self) -> nn.Parameter :

        # mode dot operation of tensorly library
        # student_param = multi_mode_dot(self.subset, modes=[2, 1, 0], matrix_or_vec_list=[self.W_i, self.W_o, self.W_l])
        student_param = multi_mode_dot(self.subset, modes=[0], matrix_or_vec_list=self.W_l)
        return self.tanh(student_param.squeeze(0)) * self.W + self.B

# new
class WeightGenerator(nn.Module):
    def __init__(
        self,
        weight_class_name: str,
        current_layer_index: int,
        num_student_layers: int,
        student_weight_in: int,
        student_weight_out: int
    ):
        super().__init__()
        self.subset = TeacherWeightGroup.generate_weight_group(
            weight_class_name, current_layer_index, num_student_layers
        )
        teacher_weight_out, teacher_weight_in, num_adjacent_layers = self.subset.size()

        self.W_l = nn.Parameter(torch.empty(num_adjacent_layers, 1))
        self.W = nn.Parameter(torch.ones(student_weight_out, student_weight_in))
        self.B = nn.Parameter(torch.zeros(student_weight_out, student_weight_in))

        self.tanh = nn.Tanh()
        self.init_weights_()
    
    def init_weights_(self):
        init.xavier_uniform_(self.W_l)

    def forward(self) -> nn.Parameter :
        student_param = self.subset.matmul(self.W_l)
        return self.tanh(student_param.squeeze(-1)) * self.W + self.B


class BiasGenerator(nn.Module):
    def __init__(
        self,
        weight_class_name: str,
        current_layer_index: int,
        num_student_layers: int,
        student_out_features: int,
    ):

        self.subset = TeacherWeightGroup.generate_weight_group(
            weight_class_name, current_layer_index, num_student_layers
        )
        num_adjacent_layers, teacher_out_features = self.subset.shape

        # self.W_o = nn.Parameter(torch.empty(student_out_features, teacher_out_features))
        self.W_l = nn.Parameter(torch.empty(1, num_adjacent_layers))
        self.W = nn.Parameter(torch.ones(student_out_features))
        self.B = nn.Parameter(torch.zeros(student_out_features))

        self.tanh = nn.Tanh()
        self.init_weights_()
    
    def init_weights_(self):
        # init.xavier_uniform_(self.W_o)
        init.xavier_uniform_(self.W_l)
    
    def forward(self) -> nn.Parameter :

        # mode dot operation of tensorly library
        # student_param = multi_mode_dot(self.subset, modes=[1, 0], matrix_or_vec_list=[self.W_o, self.W_l])
        student_param = multi_mode_dot(self.subset, modes=[0], matrix_or_vec_list=[self.W_l])

        return self.tanh(student_param.squeeze(0)) * self.W + self.B

# New
class BiasGenerator(nn.Module):
    def __init__(
        self,
        weight_class_name: str,
        current_layer_index: int,
        num_student_layers: int,
        student_out_features: int,
    ):
        super().__init__()
        self.subset = TeacherWeightGroup.generate_bias_group(
            weight_class_name, current_layer_index, num_student_layers
        )
        teacher_out_features, num_adjacent_layers = self.subset.shape

        self.W_l = nn.Parameter(torch.empty(num_adjacent_layers, 1))
        self.W = nn.Parameter(torch.ones(student_out_features))
        self.B = nn.Parameter(torch.zeros(student_out_features))

        self.tanh = nn.Tanh()
        self.init_weights_()
    
    def init_weights_(self):
        init.xavier_uniform_(self.W_l)
    
    def forward(self) -> nn.Parameter :
        student_param = self.subset.matmul(self.W_l)
        return self.tanh(student_param.squeeze(-1)) * self.W + self.B

class StudentLinear(nn.Module):
    def __init__(
        self, 
        weight_class_name: str, 
        current_layer_index: int, 
        num_student_layers: int,
        in_features: int, 
        out_features: int,
    ):
        super().__init__()

        self.weight_generator = WeightGenerator(
            weight_class_name = weight_class_name, 
            current_layer_index = current_layer_index, 
            num_student_layers = num_student_layers, 
            student_weight_in = in_features, 
            student_weight_out = out_features,
        )
        self.bias_generator = BiasGenerator(
            weight_class_name = weight_class_name, 
            current_layer_index = current_layer_index, 
            num_student_layers = num_student_layers, 
            student_out_features = out_features,
        )
            
    def forward(self, inputs: torch.Tensor) -> torch.Tensor :

        student_weight = self.weight_generator()
        student_bias = self.bias_generator()
        
        return F.linear(inputs, student_weight, student_bias)

class StudentMLP(nn.Module):
    def __init__(
        self, 
        current_layer_index,
        num_student_layers,
        config
    ):
        super().__init__()
        hidden_size = config["hidden_size"]
        intermediate_size = config["intermediate_size"]
        
        self.c_fc = StudentLinear(
            "decoder.mlp.c_fc", 
            current_layer_index, 
            num_student_layers, 
            hidden_size, 
            intermediate_size
        )
        self.c_proj = StudentLinear(
            "decoder.mlp.c_proj", 
            current_layer_index, 
            num_student_layers, 
            intermediate_size, 
            hidden_size
        )
        self.act = gelu_new
        self.dropout = nn.Dropout(config["resid_pdrop"])
    
    def forward(self, hidden_states):
        hidden_states = self.c_fc(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.c_proj(hidden_states)
        hidden_states = self.dropout(hidden_states)
        return hidden_states

# Model and Data

In [3]:
from model import GraftAttentionModule

class GrafomerModel(nn.Module, ModuleUtilsMixin, GenerationMixin, PushToHubMixin):
    def __init__(self, enc_name, dec_name):
        super().__init__()

        self.encoder = getattr(__import__("transformers"), 'AutoModel').from_pretrained(enc_name)
        self.decoder = getattr(__import__("transformers"), 'AutoModelWithLMHead').from_pretrained(dec_name)

        self.config = self.decoder.config  # for compatibility in generate method
        self.config.is_encoder_decoder = True
        self.config.decoder_start_token_id = 2
        print(self.config)

        self.decoder_body = getattr(self.decoder, 'transformer')
        self.decoder_head = getattr(self.decoder, 'lm_head')

        self.bart_config = AutoConfig.from_pretrained("facebook/bart-base")
        
        self.decoder_embed_dim = 768
        self.graft_module_config = {'num_enc_layer': 2, 'num_dec_layer': 2}
        self.graft_module = GraftAttentionModule(self.bart_config, self.graft_module_config, self.decoder_embed_dim)
    
    def get_encoder(self):
        return self.encoder
        
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        decoder_input_ids=None,
        decoder_attention_mask=None,
        head_mask=None,
        decoder_head_mask=None,
        cross_attn_head_mask=None,
        encoder_outputs=None,
        past_key_values=None,
        inputs_embeds=None,
        decoder_inputs_embeds=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):

        if encoder_outputs is None:
            encoder_outputs = self.encoder(input_ids=input_ids,
                                        attention_mask=attention_mask,
                                        head_mask=head_mask,
                                        inputs_embeds=inputs_embeds,
                                        output_attentions=output_attentions,
                                        output_hidden_states=output_hidden_states,
                                        return_dict=return_dict)
        
        # If the user passed a tuple for encoder_outputs, we wrap it in a BaseModelOutput when return_dict=True
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(last_hidden_state=encoder_outputs[0],
                                            hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                                            attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None)

        # train
        if decoder_input_ids is not None:
            decoder_outputs = self.decoder_body(input_ids=decoder_input_ids,
                                                attention_mask=decoder_attention_mask,
                                                use_cache=use_cache)

            mask = _expand_mask(attention_mask, self.dtype)
            dec_mask = _make_causal_mask(decoder_attention_mask.shape, self.dtype).to(self.device) + _expand_mask(decoder_attention_mask, self.dtype)
            cross_mask = _expand_mask(attention_mask, self.dtype, tgt_len=decoder_input_ids.shape[1])
        
        # eval
        else:
            decoder_outputs = self.decoder_body(input_ids=input_ids, use_cache=use_cache)

            bsz, sql = input_ids.shape
            mask = _expand_mask(attention_mask, self.dtype)
            dec_mask = _make_causal_mask([bsz, sql], self.dtype).to(self.device)
            cross_mask = _expand_mask(attention_mask, self.dtype, tgt_len=sql)


        encoder_hidden_state = encoder_outputs[0]
        decoder_hidden_state = decoder_outputs[0]

        graformer_hidden_state = self.graft_module(
            encoder_hidden_states=encoder_hidden_state,
            encoder_attention_mask=mask,
            decoder_hidden_states=decoder_hidden_state,
            decoder_attention_mask=dec_mask,
            cross_attention_mask=cross_mask,
            output_attentions=output_attentions,
        )

        output_hidden_states = decoder_hidden_state + graformer_hidden_state
        output_hidden_states = self.decoder_head(output_hidden_states)

        return Seq2SeqLMOutput(
            logits=output_hidden_states
        )
        
    def prepare_inputs_for_generation(self, input_ids, past=None, **kwargs):
        if past is not None:
            # print(past, type(past))
            # print(len(past))
            # print(past.shape)
            pass
        return {"input_ids": input_ids, **kwargs}


In [205]:
sample['input_ids']

tensor([[  101,   146,   112,  ...,     0,     0,     0],
        [  101,  4476,  6456,  ...,     0,     0,     0],
        [  101, 11937,  1881,  ...,     0,     0,     0],
        ...,
        [  101, 28140, 20390,  ...,     0,     0,     0],
        [  101, 10167, 18873,  ...,     0,     0,     0],
        [  101,  8425,  7298,  ...,     0,     0,     0]])

In [230]:
enc_name = 'bert-base-multilingual-cased'
dec_name = 'kykim/gpt3-kor-small_based_on_gpt2'
encoder_tokenizer = getattr(__import__("transformers"), "AutoTokenizer").from_pretrained(enc_name) # source lang
decoder_tokenizer = getattr(__import__("transformers"), "BertTokenizerFast").from_pretrained(dec_name)


model = GrafomerModel(enc_name, dec_name)
model.encoder.load_state_dict(torch.load("/opt/ml/final-project-level3-nlp-01/ko_model_checkpoint/ko_encoder_64000.pt"))
model.decoder.load_state_dict(torch.load("/opt/ml/final-project-level3-nlp-01/ko_model_checkpoint/ko_decoder_64000.pt"))
model.graft_module.load_state_dict(torch.load("/opt/ml/final-project-level3-nlp-01/ko_model_checkpoint/ko_graft_module_64000.pt"))
print(f"number of model parameters: {model.num_parameters()}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected 

GPT2Config {
  "_name_or_path": "kykim/gpt3-kor-small_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2Model"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 3,
  "decoder_start_token_id": 2,
  "embd_pdrop": 0.1,
  "eos_token_id": 3,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 42000
}

number of model parameters: 330998784


In [235]:

model.to("cuda:0")
sample = {k: v.to("cuda:0") for k, v in sample.items()}

with torch.no_grad():
                        
    # decoder_input_ids = torch.ones((cfg.train_config.batch_size, 1), dtype=torch.long, device=device) * decoder_tokenizer.bos_token_id
    generated_tokens = model.generate(sample["input_ids"], attention_mask=sample["attention_mask"], max_length=int(sample["input_ids"].shape[1] * 1.3),
                                    # decoder_input_ids=eval_batch["decoder_input_ids"], decoder_attention_mask=eval_batch["decoder_attention_mask"],
                                    pad_token_id=decoder_tokenizer.pad_token_id, eos_token_id=decoder_tokenizer.eos_token_id, bos_token_id=decoder_tokenizer.bos_token_id,
                                    do_sample=True, top_k=50, top_p=0.95, repetition_penalty=1.2)
    labels = sample["labels"]

    decoded_preds = decoder_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    decoded_labels = decoder_tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

In [ ]:
torch.cuda.empty_cache()

In [241]:
decoded_preds[:3]

['',
 '수군수군으로 출세한 그는 사경 ( ) 의 우두머리, 청의 고충을 알게 된 후 서애적객사유상공과 함께 정조약에 의해 복권된 후 서인 세도정치가 그 주역이며 조정에서 이황거 ( ) 와 같은 두 가지 조로 나눠지는 것을 보고 이백량 등 문신, 명척 ( ) 을 초빙해 변론이나 토사의 기전일문 ( ) 에서 구면분답이 일목요연치 않았다.',
 '한 것은 나약하기 짝이 없는 천하통일이고, 이차적인 것이 될 수 없는 것을 신들림 (, e. b. r. von wilsch : a site. ) 에 대한 사명감부터 가지고 있다.']

In [242]:
decoded_labels[:3]

[['나는 이것들을 나의 자유로운 시간에 써 내려갈 것입니다.'],
 ['본 고안은 반도체 패키지 몰딩공정용 몰드다이에 관한 것으로서, 구체적으로는 캐비티와 런너사이에 연결 브리지를 별도로 구비하는 반도체 패키지 몰딩공정용 몰드다이에 관한 것이다.'],
 ['또한, 청지천은 서산의 남동 측 외곽에서 발원하여 북동 측 방향에서 정남 측 방향으로 흐르며, 서산 지역을 통과하는 석림천이 남동 측 방향으로 흘러 청지천으로 유입되고 있다.']]

In [238]:
decoded_preds

['',
 '수군수군으로 출세한 그는 사경 ( ) 의 우두머리, 청의 고충을 알게 된 후 서애적객사유상공과 함께 정조약에 의해 복권된 후 서인 세도정치가 그 주역이며 조정에서 이황거 ( ) 와 같은 두 가지 조로 나눠지는 것을 보고 이백량 등 문신, 명척 ( ) 을 초빙해 변론이나 토사의 기전일문 ( ) 에서 구면분답이 일목요연치 않았다.',
 '한 것은 나약하기 짝이 없는 천하통일이고, 이차적인 것이 될 수 없는 것을 신들림 (, e. b. r. von wilsch : a site. ) 에 대한 사명감부터 가지고 있다.',
 '이 같이 밝혀진 데는 그 ~ 일례로 아따만해.',
 '.',
 '##의 고나리 ( ) 가 되어도 될 정도의 이목이 있는 것 같지만, 그 중 의병들에 대한 묘사가 일선 개찰소 및 하급자 ( 혹은 선관위원들 ) 에게까지 영향을 미친다는 것을 들어, 고나리를 극도로 노골화시키기도 한다.',
 '',
 '다른 건 다 제쳐두고서라도, 본디 저, 이 정도만 하더라도 너무 좋은 게, 그 자체일 것이오.',
 '이오시 ( 오상곤 ) 로부터 직접 위자료가 한 푼도 지급될 수 없다는 내용의 통지서를 전해들은 지모 ( 차재원 ) 에 의해 발각돼 거사를 치를 수밖에 없고, 이는 오상의 심복으로 있는 공선무 ( 엄기영 ) 를 무찌른 후 복권을 산 이채원 역시 풀어 준 것으로 전한다.',
 '',
 '고인별의 신원화, 명주희 등 동명기체 ( ) 의 불연속적 열람을 가능케 하는 것은 가부동한 이 ( 나치스이 ) 를 위해 사명된 일류작가로 하여금, 사명을 더욱 공고히 한 것이다.',
 '이백여 명의 인파로 북새통인 가운데 ( 구 한내사우회. 가 ) 도정자, 나문각에 이어 지난 4월에는 ( 사관. 의병장 ) 을 포함한 세 번째 이 같은 자결명분으로 한울이천단 ( 중 ) 과의 연대적 및 항일동지회 소속을 결정하게 됐다.',
 '',
 '. 또한, 이번을 통해 그 중 몇 명을 다른 사람들에 대한 이롭게 하는, 타인 - 예지의 장려가 아닌, 일개 고부간 ( ) 에서의 타

In [247]:
torch.cuda.empty_cache()

In [4]:
import logging

logger = logging.getLogger(__name__)
formatter = logging.Formatter("%(asctime)s [%(levelname)s]: %(message)s")
file_handler = logging.FileHandler(filename='train.log')
logger.addHandler(file_handler)


enc_name = 'bert-base-multilingual-cased'
dec_name = 'Chaewon/mnmt_decoder_en'
encoder_tokenizer = getattr(__import__("transformers"), "AutoTokenizer").from_pretrained(enc_name) # source lang
decoder_tokenizer = getattr(__import__("transformers"), "AutoTokenizer").from_pretrained(dec_name)


model = GrafomerModel(enc_name, dec_name)
model.encoder.load_state_dict(torch.load("/opt/ml/final-project-level3-nlp-01/en_model_checkpoint/en_encoder.pt"))
model.decoder.load_state_dict(torch.load("/opt/ml/final-project-level3-nlp-01/en_model_checkpoint/en_decoder.pt"))
model.graft_module.load_state_dict(torch.load("/opt/ml/final-project-level3-nlp-01/en_model_checkpoint/en_graft_module.pt"))
print(f"number of model parameters: {model.num_parameters()}")

# train_set, valid_set = load_data(cfg.train_config.data_path)
# raw_dataset = load_from_disk("/opt/ml/final-project-level3-nlp-01/ko_unified_dataset")
raw_dataset = load_from_disk("/opt/ml/final-project-level3-nlp-01/en_unified_dataset")
train_set = raw_dataset["train"]
valid_set = raw_dataset["validation"]

fn_kwargs = {'max_length': 512, 'max_target_length': 1024}
tokenized_train = train_set.map(preprocess_function_with_setting(encoder_tokenizer, decoder_tokenizer, False, False), 
                                num_proc=8, batched=True, remove_columns=train_set.column_names, fn_kwargs=fn_kwargs)
tokenized_valid = valid_set.map(preprocess_function_with_setting(encoder_tokenizer, decoder_tokenizer, False, False),
                                num_proc=8, batched=True, remove_columns=valid_set.column_names, fn_kwargs=fn_kwargs)

data_collator = CustomDataCollator(encoder_pad_token_id=encoder_tokenizer.pad_token_id, decoder_pad_token_id=decoder_tokenizer.pad_token_id)
train_dataloader = DataLoader(tokenized_train, batch_size=16, pin_memory=True,
                                shuffle=True, drop_last=True, num_workers=8, collate_fn=data_collator)
valid_dataloader = DataLoader(tokenized_valid, batch_size=16, pin_memory=True, 
                                shuffle=False, drop_last=False, num_workers=8, collate_fn=data_collator)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/envs/lightweight/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:698: FutureWarning: The class `AutoModelWithLM

GPT2Config {
  "_name_or_path": "Chaewon/mnmt_decoder_en",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 2,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 50260
}

n

Loading cached processed dataset at /opt/ml/final-project-level3-nlp-01/en_unified_dataset/train/cache-3b2cda1dfea1dbb6.arrow
Loading cached processed dataset at /opt/ml/final-project-level3-nlp-01/en_unified_dataset/train/cache-035041f7c0640943.arrow
Loading cached processed dataset at /opt/ml/final-project-level3-nlp-01/en_unified_dataset/train/cache-f7126343de2a6a00.arrow
Loading cached processed dataset at /opt/ml/final-project-level3-nlp-01/en_unified_dataset/train/cache-203a2bf9c2a88ad3.arrow
Loading cached processed dataset at /opt/ml/final-project-level3-nlp-01/en_unified_dataset/train/cache-f176d9fd74456830.arrow
Loading cached processed dataset at /opt/ml/final-project-level3-nlp-01/en_unified_dataset/train/cache-800d358b182d1466.arrow
Loading cached processed dataset at /opt/ml/final-project-level3-nlp-01/en_unified_dataset/train/cache-c984da3cfc47c2ca.arrow
Loading cached processed dataset at /opt/ml/final-project-level3-nlp-01/en_unified_dataset/train/cache-09822e63c6a6634

In [5]:
sample = next(iter(train_dataloader))

In [5]:
sample['input_ids'].size()

torch.Size([16, 64])

In [277]:
sample['decoder_input_ids'].size()

torch.Size([16, 70])

In [278]:
sample['labels'].size()

torch.Size([16, 70])

In [6]:
model.to("cuda:0")
sample = {k: v.to("cuda:0") for k, v in sample.items()}

In [279]:
model.to("cuda:0")
sample = {k: v.to("cuda:0") for k, v in sample.items()}

with torch.no_grad():
                        
    # decoder_input_ids = torch.ones((cfg.train_config.batch_size, 1), dtype=torch.long, device=device) * decoder_tokenizer.bos_token_id
    generated_tokens = model.generate(sample["input_ids"], attention_mask=sample["attention_mask"], max_length=int(sample["input_ids"].shape[1] * 1.3),
                                    # decoder_input_ids=eval_batch["decoder_input_ids"], decoder_attention_mask=eval_batch["decoder_attention_mask"],
                                    pad_token_id=decoder_tokenizer.pad_token_id, eos_token_id=decoder_tokenizer.eos_token_id, bos_token_id=decoder_tokenizer.bos_token_id,
                                    do_sample=True, top_k=50, top_p=0.95, repetition_penalty=1.2)
    labels = sample["labels"]

    decoded_preds = decoder_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    decoded_labels = decoder_tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

In [280]:
decoded_preds[:3]

['#', '#', '#']

In [281]:
decoded_labels

[['main issue faced by the Department for General Assembly and Conference Management in the past year had been the servicing of the Human Rights Council and its machinery; while it was fully committed to servicing that body, it could only plan for the workload it generates and summon the required financial and human resources to manage its conferences within the applicable rules, processes and procedures.'],
 ['iding the merits of the case, the Committee noted the lack of details provided by the complainant concerning his Falun Gong activities and a number of inconsistencies in his account of facts that had undermined the general credibility of his claims, as well as his failure to provide compelling evidence in support of his claim.'],
 ["you just got this piece of energy and you just, you're manipulating it."],
 ['performance evaluation area of the consortium joint training center consists of the items of project input, project operation, and project result.'],
 ['largest increase wa

In [273]:
del model
del sample
torch.cuda.empty_cache()

In [282]:
for name, _ in model.named_parameters():
    print(name)

encoder.embeddings.word_embeddings.weight
encoder.embeddings.position_embeddings.weight
encoder.embeddings.token_type_embeddings.weight
encoder.embeddings.LayerNorm.weight
encoder.embeddings.LayerNorm.bias
encoder.encoder.layer.0.attention.self.query.weight
encoder.encoder.layer.0.attention.self.query.bias
encoder.encoder.layer.0.attention.self.key.weight
encoder.encoder.layer.0.attention.self.key.bias
encoder.encoder.layer.0.attention.self.value.weight
encoder.encoder.layer.0.attention.self.value.bias
encoder.encoder.layer.0.attention.output.dense.weight
encoder.encoder.layer.0.attention.output.dense.bias
encoder.encoder.layer.0.attention.output.LayerNorm.weight
encoder.encoder.layer.0.attention.output.LayerNorm.bias
encoder.encoder.layer.0.intermediate.dense.weight
encoder.encoder.layer.0.intermediate.dense.bias
encoder.encoder.layer.0.output.dense.weight
encoder.encoder.layer.0.output.dense.bias
encoder.encoder.layer.0.output.LayerNorm.weight
encoder.encoder.layer.0.output.LayerNorm

In [74]:
sample = next(iter(train_dataloader))
sample.keys()

dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'])

In [75]:
sample['input_ids'].size()

torch.Size([16, 75])

In [76]:
sample['attention_mask'].size()

torch.Size([16, 75])

In [290]:
model.to("cpu")

GrafomerModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [7]:
TeacherWeightGroup.set_network(model)

# Encoder

## Config

In [462]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("bert-base-multilingual-cased")

In [463]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

In [8]:
from transformers import BertConfig

class StudencEncoderConfig(BertConfig):
    
    def __init__(
        self,
        _name_or_path="bert-base-multilingual-cased",
        architectures=["BertForMaskedLM"],
        vocab_size=119547,
        hidden_size=768,
        num_hidden_layers=4, # Student Layer Length
        num_attention_heads=12,
        intermediate_size=3072,
        hidden_act="gelu",
        hidden_dropout_prob=0.1,
        attention_probs_dropout_prob=0.1,
        max_position_embeddings=512,
        type_vocab_size=2,
        initializer_range=0.02,
        layer_norm_eps=1e-12,
        pad_token_id=0,
        position_embedding_type="absolute",
        use_cache=True,
        classifier_dropout=None,
        directionality="bidi",
        model_type="bert",
        pooler_fc_size=768,
        pooler_num_attention_heads=12,
        pooler_num_fc_layers=3,
        pooler_size_per_head=128,
        pooler_type="first_token_transform",
        transformers_version="4.12.5",
        **kwargs
    ):
        super().__init__(pad_token_id=pad_token_id, **kwargs)
        
        self._name_or_path = _name_or_path,
        self.architectures = architectures
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.hidden_act = hidden_act
        self.intermediate_size = intermediate_size
        self.hidden_dropout_prob = hidden_dropout_prob
        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.max_position_embeddings = max_position_embeddings
        self.type_vocab_size = type_vocab_size
        self.initializer_range = initializer_range
        self.layer_norm_eps = layer_norm_eps
        self.position_embedding_type = position_embedding_type
        self.use_cache = use_cache
        self.classifier_dropout = classifier_dropout
        self.directionality = directionality
        self.model_type = model_type
        self.pooler_fc_size = pooler_fc_size
        self.pooler_num_attention_heads = pooler_num_attention_heads
        self.pooler_num_fc_layers = pooler_num_fc_layers
        self.pooler_size_per_head = pooler_size_per_head
        self.pooler_type = pooler_type
        self.transformers_version = transformers_version


In [9]:
student_encoder_config = StudencEncoderConfig()

In [466]:
student_encoder_config

StudencEncoderConfig {
  "_name_or_path": [
    "bert-base-multilingual-cased"
  ],
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

## Encoder Embedding

In [10]:
class StudentBertEmbeddings(nn.Module):
    """Construct the embeddings from word, position and token_type embeddings."""

    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)

        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # position_ids (1, len position emb) is contiguous in memory and exported when serialized
        self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")
        self.register_buffer("position_ids", torch.arange(config.max_position_embeddings).expand((1, -1)))
        if version.parse(torch.__version__) > version.parse("1.6.0"):
            self.register_buffer(
                "token_type_ids",
                torch.zeros(self.position_ids.size(), dtype=torch.long, device=self.position_ids.device),
                persistent=False,
            )

    def forward(
        self, input_ids=None, token_type_ids=None, position_ids=None, inputs_embeds=None, past_key_values_length=0
    ):
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        seq_length = input_shape[1]

        if position_ids is None:
            position_ids = self.position_ids[:, past_key_values_length : seq_length + past_key_values_length]

        # Setting the token_type_ids to the registered buffer in constructor where it is all zeros, which usually occurs
        # when its auto-generated, registered buffer helps users when tracing the model without passing token_type_ids, solves
        # issue #5664
        if token_type_ids is None:
            if hasattr(self, "token_type_ids"):
                buffered_token_type_ids = self.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(input_shape[0], seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=self.position_ids.device)

        if inputs_embeds is None:
            inputs_embeds = self.word_embeddings(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        embeddings = inputs_embeds + token_type_embeddings
        if self.position_embedding_type == "absolute":
            position_embeddings = self.position_embeddings(position_ids)
            embeddings += position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings


In [13]:
student_embeddings = StudentBertEmbeddings(student_encoder_config)

In [14]:
student_embeddings(sample['input_ids']).size()

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select

## Encoder Attention

### Self Attention

In [11]:
class StudentBertSelfAttention(nn.Module):
    def __init__(self, config, current_layer_index, num_student_layers):
        super().__init__()
        if config.hidden_size % config.num_attention_heads != 0 and not hasattr(config, "embedding_size"):
            raise ValueError(
                f"The hidden size ({config.hidden_size}) is not a multiple of the number of attention "
                f"heads ({config.num_attention_heads})"
            )

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = StudentLinear(
            "encoder.attention.self.query",
            current_layer_index, 
            num_student_layers, 
            config.hidden_size, 
            self.all_head_size,
        )
        self.key = StudentLinear(
            "encoder.attention.self.key", 
            current_layer_index, 
            num_student_layers, 
            config.hidden_size, 
            self.all_head_size,
        )
        self.value = StudentLinear(
            "encoder.attention.self.value", 
            current_layer_index, 
            num_student_layers, 
            config.hidden_size, 
            self.all_head_size,
        )

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
        self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")
        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            self.max_position_embeddings = config.max_position_embeddings
            self.distance_embedding = nn.Embedding(2 * config.max_position_embeddings - 1, self.attention_head_size)

        self.is_decoder = config.is_decoder

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_value=None,
        output_attentions=False,
    ):
        mixed_query_layer = self.query(hidden_states)

        # If this is instantiated as a cross-attention module, the keys
        # and values come from an encoder; the attention mask needs to be
        # such that the encoder's padding tokens are not attended to.
        is_cross_attention = encoder_hidden_states is not None

        if is_cross_attention and past_key_value is not None:
            # reuse k,v, cross_attentions
            key_layer = past_key_value[0]
            value_layer = past_key_value[1]
            attention_mask = encoder_attention_mask
        elif is_cross_attention:
            key_layer = self.transpose_for_scores(self.key(encoder_hidden_states))
            value_layer = self.transpose_for_scores(self.value(encoder_hidden_states))
            attention_mask = encoder_attention_mask
        elif past_key_value is not None:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))
            key_layer = torch.cat([past_key_value[0], key_layer], dim=2)
            value_layer = torch.cat([past_key_value[1], value_layer], dim=2)
        else:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))

        query_layer = self.transpose_for_scores(mixed_query_layer)

        if self.is_decoder:
            # if cross_attention save Tuple(torch.Tensor, torch.Tensor) of all cross attention key/value_states.
            # Further calls to cross_attention layer can then reuse all cross-attention
            # key/value_states (first "if" case)
            # if uni-directional self-attention (decoder) save Tuple(torch.Tensor, torch.Tensor) of
            # all previous decoder key/value_states. Further calls to uni-directional self-attention
            # can concat previous decoder key/value_states to current projected key/value_states (third "elif" case)
            # if encoder bi-directional self-attention `past_key_value` is always `None`
            past_key_value = (key_layer, value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            seq_length = hidden_states.size()[1]
            position_ids_l = torch.arange(seq_length, dtype=torch.long, device=hidden_states.device).view(-1, 1)
            position_ids_r = torch.arange(seq_length, dtype=torch.long, device=hidden_states.device).view(1, -1)
            distance = position_ids_l - position_ids_r
            positional_embedding = self.distance_embedding(distance + self.max_position_embeddings - 1)
            positional_embedding = positional_embedding.to(dtype=query_layer.dtype)  # fp16 compatibility

            if self.position_embedding_type == "relative_key":
                relative_position_scores = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores
            elif self.position_embedding_type == "relative_key_query":
                relative_position_scores_query = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                relative_position_scores_key = torch.einsum("bhrd,lrd->bhlr", key_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores_query + relative_position_scores_key

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        if self.is_decoder:
            outputs = outputs + (past_key_value,)
        return outputs


In [471]:
StudentBertSelfAttention(student_encoder_config, 1, 2)

StudentBertSelfAttention(
  (query): StudentLinear(
    (weight_generator): WeightGenerator(
      (tanh): Tanh()
    )
    (bias_generator): BiasGenerator(
      (tanh): Tanh()
    )
  )
  (key): StudentLinear(
    (weight_generator): WeightGenerator(
      (tanh): Tanh()
    )
    (bias_generator): BiasGenerator(
      (tanh): Tanh()
    )
  )
  (value): StudentLinear(
    (weight_generator): WeightGenerator(
      (tanh): Tanh()
    )
    (bias_generator): BiasGenerator(
      (tanh): Tanh()
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
)

### Self Output

In [12]:
class StudentBertSelfOutput(nn.Module):
    def __init__(self, config, current_layer_index, num_student_layers):
        super().__init__()
        
        self.dense = StudentLinear(
            "encoder.attention.output.dense",
            current_layer_index, 
            num_student_layers, 
            config.hidden_size, 
            config.hidden_size,
        )
        
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states


In [473]:
StudentBertSelfOutput(student_encoder_config, 1, 2)

StudentBertSelfOutput(
  (dense): StudentLinear(
    (weight_generator): WeightGenerator(
      (tanh): Tanh()
    )
    (bias_generator): BiasGenerator(
      (tanh): Tanh()
    )
  )
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

### Attention (self attention + self output)

In [13]:
class StudentBertAttention(nn.Module):
    def __init__(self, config, current_layer_index, num_student_layers):
        super().__init__()
        self.self = StudentBertSelfAttention(config, current_layer_index, num_student_layers)
        self.output = StudentBertSelfOutput(config, current_layer_index, num_student_layers)
        self.pruned_heads = set()

    def prune_heads(self, heads):
        if len(heads) == 0:
            return
        heads, index = find_pruneable_heads_and_indices(
            heads, self.self.num_attention_heads, self.self.attention_head_size, self.pruned_heads
        )

        # Prune linear layers
        self.self.query = prune_linear_layer(self.self.query, index)
        self.self.key = prune_linear_layer(self.self.key, index)
        self.self.value = prune_linear_layer(self.self.value, index)
        self.output.dense = prune_linear_layer(self.output.dense, index, dim=1)

        # Update hyper params and store pruned heads
        self.self.num_attention_heads = self.self.num_attention_heads - len(heads)
        self.self.all_head_size = self.self.attention_head_size * self.self.num_attention_heads
        self.pruned_heads = self.pruned_heads.union(heads)

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_value=None,
        output_attentions=False,
    ):
        self_outputs = self.self(
            hidden_states,
            attention_mask,
            head_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            past_key_value,
            output_attentions,
        )
        attention_output = self.output(self_outputs[0], hidden_states)
        outputs = (attention_output,) + self_outputs[1:]  # add attentions if we output them
        return outputs

In [475]:
for name, _ in StudentBertAttention(student_encoder_config, 1, 2).named_parameters():
    print(name)

self.query.weight_generator.W_l
self.query.weight_generator.W
self.query.weight_generator.B
self.query.bias_generator.W_l
self.query.bias_generator.W
self.query.bias_generator.B
self.key.weight_generator.W_l
self.key.weight_generator.W
self.key.weight_generator.B
self.key.bias_generator.W_l
self.key.bias_generator.W
self.key.bias_generator.B
self.value.weight_generator.W_l
self.value.weight_generator.W
self.value.weight_generator.B
self.value.bias_generator.W_l
self.value.bias_generator.W
self.value.bias_generator.B
output.dense.weight_generator.W_l
output.dense.weight_generator.W
output.dense.weight_generator.B
output.dense.bias_generator.W_l
output.dense.bias_generator.W
output.dense.bias_generator.B
output.LayerNorm.weight
output.LayerNorm.bias


## Encoder FFN

### Intermediate, fc1

In [14]:
class StudentBertIntermediate(nn.Module):
    def __init__(self, config, current_layer_index, num_student_layers):
        super().__init__()

        self.dense = StudentLinear(
            "encoder.intermediate.dense",
            current_layer_index, 
            num_student_layers, 
            config.hidden_size, 
            config.intermediate_size,
        )

        if isinstance(config.hidden_act, str):
            self.intermediate_act_fn = nn.functional.gelu
        else:
            self.intermediate_act_fn = config.hidden_act

    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states


### Output, fc2

In [15]:
class StudentBertOutput(nn.Module):
    def __init__(self, config, current_layer_index, num_student_layers):
        super().__init__()
        
        self.dense = StudentLinear(
            "encoder.output.dense",
            current_layer_index, 
            num_student_layers, 
            config.intermediate_size,
            config.hidden_size,
        )
        
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

## Encoder Layer

In [16]:
class StudentBertLayer(nn.Module):
    def __init__(self, config, current_layer_index, num_student_layers):
        super().__init__()
        self.chunk_size_feed_forward = config.chunk_size_feed_forward
        self.seq_len_dim = 1
        self.attention = StudentBertAttention(config, current_layer_index, num_student_layers)
        self.is_decoder = config.is_decoder
        self.add_cross_attention = config.add_cross_attention
        if self.add_cross_attention:
            assert self.is_decoder, f"{self} should be used as a decoder model if cross attention is added"
            self.crossattention = StudentBertAttention(config, current_layer_index, num_student_layers)
        self.intermediate = StudentBertIntermediate(config, current_layer_index, num_student_layers)
        self.output = StudentBertOutput(config, current_layer_index, num_student_layers)

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_value=None,
        output_attentions=False,
    ):
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attn_past_key_value = past_key_value[:2] if past_key_value is not None else None
        self_attention_outputs = self.attention(
            hidden_states,
            attention_mask,
            head_mask,
            output_attentions=output_attentions,
            past_key_value=self_attn_past_key_value,
        )
        attention_output = self_attention_outputs[0]

        # if decoder, the last output is tuple of self-attn cache
        if self.is_decoder:
            outputs = self_attention_outputs[1:-1]
            present_key_value = self_attention_outputs[-1]
        else:
            outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights

        cross_attn_present_key_value = None
        if self.is_decoder and encoder_hidden_states is not None:
            assert hasattr(
                self, "crossattention"
            ), f"If `encoder_hidden_states` are passed, {self} has to be instantiated with cross-attention layers by setting `config.add_cross_attention=True`"

            # cross_attn cached key/values tuple is at positions 3,4 of past_key_value tuple
            cross_attn_past_key_value = past_key_value[-2:] if past_key_value is not None else None
            cross_attention_outputs = self.crossattention(
                attention_output,
                attention_mask,
                head_mask,
                encoder_hidden_states,
                encoder_attention_mask,
                cross_attn_past_key_value,
                output_attentions,
            )
            attention_output = cross_attention_outputs[0]
            outputs = outputs + cross_attention_outputs[1:-1]  # add cross attentions if we output attention weights

            # add cross-attn cache to positions 3,4 of present_key_value tuple
            cross_attn_present_key_value = cross_attention_outputs[-1]
            present_key_value = present_key_value + cross_attn_present_key_value

        layer_output = apply_chunking_to_forward(
            self.feed_forward_chunk, self.chunk_size_feed_forward, self.seq_len_dim, attention_output
        )
        outputs = (layer_output,) + outputs

        # if decoder, return the attn key/values as the last output
        if self.is_decoder:
            outputs = outputs + (present_key_value,)

        return outputs

    def feed_forward_chunk(self, attention_output):
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output


In [479]:
for name, _ in StudentBertLayer(student_encoder_config, 1, 2).named_parameters():
    print(name)

attention.self.query.weight_generator.W_l
attention.self.query.weight_generator.W
attention.self.query.weight_generator.B
attention.self.query.bias_generator.W_l
attention.self.query.bias_generator.W
attention.self.query.bias_generator.B
attention.self.key.weight_generator.W_l
attention.self.key.weight_generator.W
attention.self.key.weight_generator.B
attention.self.key.bias_generator.W_l
attention.self.key.bias_generator.W
attention.self.key.bias_generator.B
attention.self.value.weight_generator.W_l
attention.self.value.weight_generator.W
attention.self.value.weight_generator.B
attention.self.value.bias_generator.W_l
attention.self.value.bias_generator.W
attention.self.value.bias_generator.B
attention.output.dense.weight_generator.W_l
attention.output.dense.weight_generator.W
attention.output.dense.weight_generator.B
attention.output.dense.bias_generator.W_l
attention.output.dense.bias_generator.W
attention.output.dense.bias_generator.B
attention.output.LayerNorm.weight
attention.outp

## Encoder

In [17]:
class StudentBertEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([StudentBertLayer(config, layer_index, config.num_hidden_layers+1) for layer_index in range(1, config.num_hidden_layers+1)])
        self.gradient_checkpointing = False

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_values=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=True,
    ):
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

        next_decoder_cache = () if use_cache else None
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            past_key_value = past_key_values[i] if past_key_values is not None else None

            if self.gradient_checkpointing and self.training:

                if use_cache:
                    logger.warning(
                        "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                    )
                    use_cache = False

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs, past_key_value, output_attentions)

                    return custom_forward

                layer_outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(layer_module),
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                )
            else:
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
                )

            hidden_states = layer_outputs[0]
            if use_cache:
                next_decoder_cache += (layer_outputs[-1],)
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        # if not return_dict:
        #     return tuple(
        #         v
        #         for v in [
        #             hidden_states,
        #             next_decoder_cache,
        #             all_hidden_states,
        #             all_self_attentions,
        #             all_cross_attentions,
        #         ]
        #         if v is not None
        #     )
        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=next_decoder_cache,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )


In [605]:
StudentBertEncoder(student_encoder_config)

StudentBertEncoder(
  (layer): ModuleList(
    (0): StudentBertLayer(
      (attention): StudentBertAttention(
        (self): StudentBertSelfAttention(
          (query): StudentLinear(
            (weight_generator): WeightGenerator(
              (tanh): Tanh()
            )
            (bias_generator): BiasGenerator(
              (tanh): Tanh()
            )
          )
          (key): StudentLinear(
            (weight_generator): WeightGenerator(
              (tanh): Tanh()
            )
            (bias_generator): BiasGenerator(
              (tanh): Tanh()
            )
          )
          (value): StudentLinear(
            (weight_generator): WeightGenerator(
              (tanh): Tanh()
            )
            (bias_generator): BiasGenerator(
              (tanh): Tanh()
            )
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): StudentBertSelfOutput(
          (dense): StudentLinear(
            (weight_generator): Wei

## Encoder Pooler

In [18]:
class StudentBertPooler(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = StudentLinear("encoder.pooler.dense", 1, 1, config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output


In [607]:
# 차원을 바꾸지 않는다면 teacher model의 pooler를 그대로 쓰는게 좋을 것 같아요
TeacherWeightGroup.teacher_model.encoder.pooler

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

In [608]:
test = torch.rand(16, 71, 768)

teacher_pooler = TeacherWeightGroup.teacher_model.encoder.pooler
teacher_pooler(test).size()

torch.Size([16, 768])

In [609]:
TeacherWeightGroup.teacher_model.encoder.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(119547, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [610]:
TeacherWeightGroup.teacher_model.encoder.embeddings(sample['input_ids']).size()

torch.Size([16, 125, 768])

## Encoder Model

In [19]:
class StudentBertModel(nn.Module, ModuleUtilsMixin, GenerationMixin, PushToHubMixin):
    def __init__(self, config, add_pooling_layer=True):
        super().__init__()
        self.config = config

        # self.embeddings = StudentBertEmbeddings(config)
        self.embeddings = TeacherWeightGroup.teacher_model.encoder.embeddings
        self.encoder = StudentBertEncoder(config)

        self.pooler = TeacherWeightGroup.teacher_model.encoder.pooler if add_pooling_layer else None

        self.init_weights()

    def init_weights(self):

        self.apply(self._init_weights)
        # self.tie_weigths()
    
    def _init_weights(self, module):
        """Initialize the weights"""
        # teacher model로부터 불러온 가중치를 초기화하면 안됨 -> nn.Embedding 부분 주석 처리
        # StudentLinear는 nn.Linear 인스턴스가 아니므로 초기화되지 않음
        # Pooler는 초기화될수도.. -> nn.Linear 부분 주석 처리
        # if isinstance(module, nn.Linear):
        #     # Slightly different from the TF version which uses truncated_normal for initialization
        #     # cf https://github.com/pytorch/pytorch/pull/5617
        #     module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
        #     if module.bias is not None:
        #         module.bias.data.zero_()
        if isinstance(module, nn.Embedding):
            # module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value
    
    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer} See base
        class PreTrainedModel
        """
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)
    
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_values=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        if input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")
        
        batch_size, seq_length = input_shape
        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)

        if token_type_ids is None:
            if hasattr(self.embeddings, "token_type_ids"):
                buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)
        
        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape, device)

        # (is_decoder... skip ..)
        encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        # if not return_dict:
        #     return (sequence_output, pooled_output) + encoder_outputs[1:]

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )


In [20]:
student_encoder = StudentBertModel(student_encoder_config)

In [35]:
encoder_output = student_encoder(sample['input_ids'], sample['attention_mask'])

In [36]:
encoder_output.last_hidden_state.size()

torch.Size([16, 59, 768])

In [615]:
encoder_output.hidden_states[-1].size()

torch.Size([16, 125, 768])

In [616]:
encoder_output.attentions[0].size()

torch.Size([16, 12, 125, 125])

In [617]:
encoder_output.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states', 'attentions'])

In [603]:
len(encoder_output)

4

In [618]:
encoder_output[0].size()

torch.Size([16, 125, 768])

In [619]:
encoder_output.pooler_output.size()

torch.Size([16, 768])

In [620]:
print(encoder_output[0].size())
print(encoder_output[1].size())

torch.Size([16, 125, 768])
torch.Size([16, 768])


In [408]:
from transformers import BertTokenizer #, BertModel

test_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertModel.from_pretrained("bert-base-multilingual-cased")
test_text = "Replace me by any text you'd like."
encoded_input = test_tokenizer(test_text, return_tensors='pt')
output = student_encoder(**encoded_input)

In [409]:
encoded_input

{'input_ids': tensor([[  101, 72337, 72654, 10911, 10155, 11178, 15541, 13028,   112,   172,
         11850,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [410]:
len(encoded_input['input_ids'][0])

13

In [412]:
print(output[0].size())
print(output[1].size())
print(output[2][-1].size())

torch.Size([1, 13, 768])
torch.Size([1, 768])
torch.Size([1, 13, 768])


In [115]:
from transformers import BertModel

test_encoder = BertModel.from_pretrained("bert-base-multilingual-cased")
encoded_input = test_tokenizer(test_text, return_tensors='pt')
output = test_encoder(**encoded_input)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [189]:
print(output[0].size())
print(output[1].size())

torch.Size([1, 13, 768])
torch.Size([1, 768])


In [116]:
student_encoder

StudentBertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): StudentBertEncoder(
    (layer): ModuleList(
      (0): StudentBertLayer(
        (attention): StudentBertAttention(
          (self): StudentBertSelfAttention(
            (query): StudentLinear(
              (weight_generator): WeightGenerator(
                (tanh): Tanh()
              )
              (bias_generator): BiasGenerator(
                (tanh): Tanh()
              )
            )
            (key): StudentLinear(
              (weight_generator): WeightGenerator(
                (tanh): Tanh()
              )
              (bias_generator): BiasGenerator(
                (tanh): Tanh()
              )
            )
            (value)

In [119]:
print(f"number of teacher encoder parameters: {TeacherWeightGroup.teacher_model.encoder.num_parameters()}")
print(f"number of student encoder parameters: {student_encoder.num_parameters()}")

number of teacher encoder parameters: 177853440
number of student encoder parameters: 149489760


# Graft Module

In [120]:
TeacherWeightGroup.teacher_model.graft_module

GraftAttentionModule(
  (graft_encoder): ModuleList(
    (0): BartEncoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=768, out_features=3072, bias=True)
      (fc2): Linear(in_features=3072, out_features=768, bias=True)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (1): BartEncoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_p

# Decoder

In [491]:
TeacherWeightGroup.teacher_model.decoder

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


## Config

In [37]:
TeacherWeightGroup.teacher_model.decoder.config

GPT2Config {
  "_name_or_path": "Chaewon/mnmt_decoder_en",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 2,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 50260
}

In [21]:
from transformers import GPT2Config

In [22]:
config = GPT2Config()

In [23]:
config

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 50257
}

In [24]:
class StudentDecoderConfig(GPT2Config):
    
    def __init__(
        self,
        _name_or_path="kykim/gpt3-kor-small_based_on_gpt2",
        architectures=["GPT2Model"],
        vocab_size=42000,
        n_positions=2048,
        n_ctx=2048,
        n_embd=768,
        n_layer=4, # = num_hidden_layers = num_student_layers
        n_head=12,
        n_inner=None,
        activation_function="gelu_new",
        resid_pdrop=0.1,
        embd_pdrop=0.1,
        attn_pdrop=0.1,
        layer_norm_epsilon=1e-05,
        initializer_range=0.02,
        summary_type="cls_index",
        summary_use_proj=True,
        summary_activation=None,
        summary_proj_to_labels=True,
        summary_first_dropout=0.1,
        scale_attn_weights=True,
        use_cache=True,
        bos_token_id=3,
        eos_token_id=3,
        transformers_version="4.13.0",
        scale_attn_by_inverse_layer_idx=False,
        reorder_and_upcast_attn=False,
        gradient_checkpointing=False,
        decoder_start_token_id=2,
        is_encoder_decoder=True,
        model_type="gpt2",
        **kwargs
    ):
        super().__init__(bos_token_id=bos_token_id, eos_token_id=eos_token_id, **kwargs)
        
        self.vocab_size = vocab_size
        self.n_ctx = n_ctx
        self.n_positions = n_positions
        self.n_embd = n_embd
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_inner = n_inner
        self.activation_function = activation_function
        self.resid_pdrop = resid_pdrop
        self.embd_pdrop = embd_pdrop
        self.attn_pdrop = attn_pdrop
        self.layer_norm_epsilon = layer_norm_epsilon
        self.initializer_range = initializer_range
        self.summary_type = summary_type
        self.summary_use_proj = summary_use_proj
        self.summary_activation = summary_activation
        self.summary_first_dropout = summary_first_dropout
        self.summary_proj_to_labels = summary_proj_to_labels
        self.scale_attn_weights = scale_attn_weights
        self.use_cache = use_cache
        self.bos_token_id = bos_token_id
        self.eos_token_id = eos_token_id
        self.transformers_version = transformers_version,
        self.scale_attn_by_inverse_layer_idx = scale_attn_by_inverse_layer_idx,
        self.reorder_and_upcast_attn = reorder_and_upcast_attn,
        self.gradient_checkpointing = gradient_checkpointing,
        self.decoder_start_token_id = decoder_start_token_id,
        self.architectures = architectures,
        self._name_or_path = _name_or_path,
        self.is_encoder_decoder = is_encoder_decoder,
        self.model_type = model_type,

        


In [25]:
student_decoder_config = StudentDecoderConfig()

In [43]:
student_decoder_config._name_or_path[0]

'kykim/gpt3-kor-small_based_on_gpt2'

In [499]:
student_decoder_config

StudentDecoderConfig {
  "_name_or_path": [
    "kykim/gpt3-kor-small_based_on_gpt2"
  ],
  "activation_function": "gelu_new",
  "architectures": [
    [
      "GPT2Model"
    ]
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 3,
  "decoder_start_token_id": [
    2
  ],
  "embd_pdrop": 0.1,
  "eos_token_id": 3,
  "gradient_checkpointing": [
    false
  ],
  "initializer_range": 0.02,
  "is_encoder_decoder": [
    true
  ],
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 4,
  "n_positions": 2048,
  "reorder_and_upcast_attn": [
    false
  ],
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": [
    false
  ],
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 42000
}

## Decoder Attention

In [44]:
print(student_decoder_config.max_position_embeddings)
print(student_decoder_config.hidden_size)
print(student_decoder_config.num_attention_heads)
print(student_decoder_config.scale_attn_weights)
print(student_decoder_config.attn_pdrop)
print(student_decoder_config.resid_pdrop)

2048
768
12
True
0.1
0.1


In [26]:
class StudentGPT2Attention(nn.Module):
    def __init__(self, config, current_layer_index, num_student_layers):
        super().__init__()

        max_positions = config.max_position_embeddings
        self.register_buffer(
            "bias",
            torch.tril(torch.ones((max_positions, max_positions), dtype=torch.uint8)).view(
                1, 1, max_positions, max_positions
            ),
        )
        self.register_buffer("masked_bias", torch.tensor(-1e4))

        self.embed_dim = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.embed_dim // self.num_heads
        self.split_size = self.embed_dim
        if self.head_dim * self.num_heads != self.embed_dim:
            raise ValueError(
                f"`embed_dim` must be divisible by num_heads (got `embed_dim`: {self.embed_dim} and `num_heads`: {self.num_heads})."
            )

        self.scale_attn_weights = config.scale_attn_weights

        self.c_attn = StudentLinear(
            "decoder.attn.c_attn", 
            current_layer_index, 
            num_student_layers, 
            self.embed_dim, 
            3 * self.embed_dim
        )
        self.c_proj = StudentLinear(
            "decoder.attn.c_proj", 
            current_layer_index, 
            num_student_layers, 
            self.embed_dim, 
            self.embed_dim
        )

        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)

        self.pruned_heads = set()

    def prune_heads(self, heads):
        pass

    def _attn(self, query, key, value, attention_mask=None, head_mask=None):
        attn_weights = torch.matmul(query, key.transpose(-1, -2))

        if self.scale_attn_weights:
            attn_weights = attn_weights / (float(value.size(-1)) ** 0.5)

        query_length, key_length = query.size(-2), key.size(-2)
        causal_mask = self.bias[:, :, key_length - query_length : key_length, :key_length].bool()
        attn_weights = torch.where(causal_mask, attn_weights, self.masked_bias.to(attn_weights.dtype))

        if attention_mask is not None:
            # Apply the attention mask
            attn_weights = attn_weights + attention_mask

        attn_weights = nn.Softmax(dim=-1)(attn_weights)
        attn_weights = self.attn_dropout(attn_weights)

        # Mask heads if we want to
        if head_mask is not None:
            attn_weights = attn_weights * head_mask

        attn_output = torch.matmul(attn_weights, value)

        return attn_output, attn_weights

    def _split_heads(self, tensor, num_heads, attn_head_size):
        """
        Splits hidden_size dim into attn_head_size and num_heads
        """
        new_shape = tensor.size()[:-1] + (num_heads, attn_head_size)
        tensor = tensor.view(*new_shape)
        return tensor.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

    def _merge_heads(self, tensor, num_heads, attn_head_size):
        """
        Merges attn_head_size dim and num_attn_heads dim into hidden_size
        """
        tensor = tensor.permute(0, 2, 1, 3).contiguous()
        new_shape = tensor.size()[:-2] + (num_heads * attn_head_size,)
        return tensor.view(new_shape)

    def forward(
        self,
        hidden_states,
        layer_past=None,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        use_cache=False,
        output_attentions=False,
    ):
        if encoder_hidden_states is not None:
            if not hasattr(self, "q_attn"):
                raise ValueError(
                    "If class is used as cross attention, the weights `q_attn` have to be defined. "
                    "Please make sure to instantiate class with `GPT2Attention(..., is_cross_attention=True)`."
                )

            query = self.q_attn(hidden_states)
            key, value = self.c_attn(encoder_hidden_states).split(self.split_size, dim=2)
            attention_mask = encoder_attention_mask
        else:
            query, key, value = self.c_attn(hidden_states).split(self.split_size, dim=2)

        query = self._split_heads(query, self.num_heads, self.head_dim)
        key = self._split_heads(key, self.num_heads, self.head_dim)
        value = self._split_heads(value, self.num_heads, self.head_dim)

        if layer_past is not None:
            past_key, past_value = layer_past
            key = torch.cat((past_key, key), dim=-2)
            value = torch.cat((past_value, value), dim=-2)

        if use_cache is True:
            present = (key, value)
        else:
            present = None

        attn_output, attn_weights = self._attn(query, key, value, attention_mask, head_mask)

        attn_output = self._merge_heads(attn_output, self.num_heads, self.head_dim)
        attn_output = self.c_proj(attn_output)
        attn_output = self.resid_dropout(attn_output)

        outputs = (attn_output, present)
        if output_attentions:
            outputs += (attn_weights,)

        return outputs  # a, present, (attentions)


In [46]:
StudentGPT2Attention(student_decoder_config, 1, 2)

StudentGPT2Attention(
  (c_attn): StudentLinear(
    (weight_generator): WeightGenerator(
      (tanh): Tanh()
    )
    (bias_generator): BiasGenerator(
      (tanh): Tanh()
    )
  )
  (c_proj): StudentLinear(
    (weight_generator): WeightGenerator(
      (tanh): Tanh()
    )
    (bias_generator): BiasGenerator(
      (tanh): Tanh()
    )
  )
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)

In [47]:
for name, _ in TeacherWeightGroup.teacher_model.decoder.named_parameters():
    print(name)

transformer.wte.weight
transformer.wpe.weight
transformer.h.0.ln_1.weight
transformer.h.0.ln_1.bias
transformer.h.0.attn.c_attn.weight
transformer.h.0.attn.c_attn.bias
transformer.h.0.attn.c_proj.weight
transformer.h.0.attn.c_proj.bias
transformer.h.0.ln_2.weight
transformer.h.0.ln_2.bias
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_fc.bias
transformer.h.0.mlp.c_proj.weight
transformer.h.0.mlp.c_proj.bias
transformer.h.1.ln_1.weight
transformer.h.1.ln_1.bias
transformer.h.1.attn.c_attn.weight
transformer.h.1.attn.c_attn.bias
transformer.h.1.attn.c_proj.weight
transformer.h.1.attn.c_proj.bias
transformer.h.1.ln_2.weight
transformer.h.1.ln_2.bias
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_fc.bias
transformer.h.1.mlp.c_proj.weight
transformer.h.1.mlp.c_proj.bias
transformer.h.2.ln_1.weight
transformer.h.2.ln_1.bias
transformer.h.2.attn.c_attn.weight
transformer.h.2.attn.c_attn.bias
transformer.h.2.attn.c_proj.weight
transformer.h.2.attn.c_proj.bias
transformer.h.2.ln_2

## Decoder MLP

In [48]:
print(student_decoder_config.n_inner)
print(student_decoder_config.hidden_size)
print(student_decoder_config.resid_pdrop)

None
768
0.1


In [27]:
class StudentGPT2MLP(nn.Module):
    def __init__(self,
                 intermediate_size,
                 config,
                 current_layer_index,
                 num_student_layers
    ):
        super().__init__()
        embed_dim = config.hidden_size

        self.c_fc = StudentLinear(
            "decoder.mlp.c_fc", 
            current_layer_index, 
            num_student_layers, 
            embed_dim, 
            intermediate_size
        )
        self.c_proj = StudentLinear(
            "decoder.mlp.c_proj", 
            current_layer_index, 
            num_student_layers, 
            intermediate_size, 
            embed_dim
        )

        self.act = gelu_new
        self.dropout = nn.Dropout(config.resid_pdrop)

    def forward(self, hidden_states):
        hidden_states = self.c_fc(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.c_proj(hidden_states)
        hidden_states = self.dropout(hidden_states)
        return hidden_states

## Decoder Block

In [50]:
print(student_decoder_config.n_inner)
print(student_decoder_config.hidden_size)
print(student_decoder_config.resid_pdrop)
print(student_decoder_config.layer_norm_epsilon)

None
768
0.1
1e-05


In [28]:
class StudentGPT2Block(nn.Module):
    def __init__(self,
                 config,
                 current_layer_index: int,
                 num_student_layers: int
    ):
        super().__init__()
        hidden_size = config.hidden_size
        inner_dim = config.n_inner if config.n_inner is not None else 4 * hidden_size

        self.ln_1 = nn.LayerNorm(hidden_size, eps=config.layer_norm_epsilon)
        self.attn = StudentGPT2Attention(config, current_layer_index, num_student_layers)
        self.ln_2 = nn.LayerNorm(hidden_size, eps=config.layer_norm_epsilon)

        # if config.add_cross_attention:
        #     self.crossattention = GPT2Attention(config, is_cross_attention=True)
        #     self.ln_cross_attn = nn.LayerNorm(hidden_size, eps=config.layer_norm_epsilon)

        self.mlp = StudentGPT2MLP(inner_dim, config, current_layer_index, num_student_layers)

    def forward(
        self,
        hidden_states,
        layer_past=None,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        use_cache=False,
        output_attentions=False,
    ):
        residual = hidden_states
        hidden_states = self.ln_1(hidden_states)
        attn_outputs = self.attn(
            hidden_states,
            layer_past=layer_past,
            attention_mask=attention_mask,
            head_mask=head_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
        )
        attn_output = attn_outputs[0]  # output_attn: a, present, (attentions)
        outputs = attn_outputs[1:]
        # residual connection
        hidden_states = attn_output + residual

        # if encoder_hidden_states is not None:
        #     # add one self-attention block for cross-attention
        #     if not hasattr(self, "crossattention"):
        #         raise ValueError(
        #             f"If `encoder_hidden_states` are passed, {self} has to be instantiated with "
        #             "cross-attention layers by setting `config.add_cross_attention=True`"
        #         )
        #     residual = hidden_states
        #     hidden_states = self.ln_cross_attn(hidden_states)
        #     cross_attn_outputs = self.crossattention(
        #         hidden_states,
        #         attention_mask=attention_mask,
        #         head_mask=head_mask,
        #         encoder_hidden_states=encoder_hidden_states,
        #         encoder_attention_mask=encoder_attention_mask,
        #         output_attentions=output_attentions,
        #     )
        #     attn_output = cross_attn_outputs[0]
        #     # residual connection
        #     hidden_states = residual + attn_output
        #     outputs = outputs + cross_attn_outputs[2:]  # add cross attentions if we output attention weights

        residual = hidden_states
        hidden_states = self.ln_2(hidden_states)
        feed_forward_hidden_states = self.mlp(hidden_states)
        # residual connection
        hidden_states = residual + feed_forward_hidden_states

        if use_cache:
            outputs = (hidden_states,) + outputs
        else:
            outputs = (hidden_states,) + outputs[1:]

        return outputs  # hidden_states, present, (attentions, cross_attentions)


In [52]:
StudentGPT2Block(student_decoder_config, 1, 2)

StudentGPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): StudentGPT2Attention(
    (c_attn): StudentLinear(
      (weight_generator): WeightGenerator(
        (tanh): Tanh()
      )
      (bias_generator): BiasGenerator(
        (tanh): Tanh()
      )
    )
    (c_proj): StudentLinear(
      (weight_generator): WeightGenerator(
        (tanh): Tanh()
      )
      (bias_generator): BiasGenerator(
        (tanh): Tanh()
      )
    )
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): StudentGPT2MLP(
    (c_fc): StudentLinear(
      (weight_generator): WeightGenerator(
        (tanh): Tanh()
      )
      (bias_generator): BiasGenerator(
        (tanh): Tanh()
      )
    )
    (c_proj): StudentLinear(
      (weight_generator): WeightGenerator(
        (tanh): Tanh()
      )
      (bias_generator): BiasGenerator(
        (tanh):

## Decoder Model

In [32]:
TeacherWeightGroup.teacher_model.decoder.transformer.wte

Embedding(50260, 768)

In [33]:
TeacherWeightGroup.teacher_model.decoder.transformer.wpe

Embedding(1024, 768)

In [55]:
print(student_decoder_config.hidden_size)
print(student_decoder_config.vocab_size)
print(student_decoder_config.max_position_embeddings)
print(student_decoder_config.embd_pdrop)
print(student_decoder_config.num_hidden_layers)
print(student_decoder_config.layer_norm_epsilon)

768
42000
2048
0.1
4
1e-05


In [29]:
class StudentGPT2Model(nn.Module, ModuleUtilsMixin, GenerationMixin, PushToHubMixin):
    # _keys_to_ignore_on_load_missing = ["attn.masked_bias"]

    def __init__(self, config):
        super().__init__()#config)
        self.config = config
        self.embed_dim = config.hidden_size

        # self.wte = nn.Embedding(config.vocab_size, self.embed_dim)
        # self.wpe = nn.Embedding(config.max_position_embeddings, self.embed_dim)

        # teacher model과 차원이 같다면
        self.wte = TeacherWeightGroup.teacher_model.decoder.transformer.wte
        self.wpe = TeacherWeightGroup.teacher_model.decoder.transformer.wpe

        self.drop = nn.Dropout(config.embd_pdrop)
        self.h = nn.ModuleList([StudentGPT2Block(config, layer_index, config.num_hidden_layers+1) for layer_index in range(1, config.num_hidden_layers+1)])
        self.ln_f = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_epsilon)

        # self.init_weights()

        # Model parallel
        self.model_parallel = False
        self.device_map = None
        self.gradient_checkpointing = False

    def get_input_embeddings(self):
        return self.wte

    def set_input_embeddings(self, new_embeddings):
        self.wte = new_embeddings

    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer}
        """
        for layer, heads in heads_to_prune.items():
            self.h[layer].attn.prune_heads(heads)

    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        use_cache=None,
        output_attentions=True,
        output_hidden_states=True,
        return_dict=None,
    ):
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
            input_ids = input_ids.view(-1, input_shape[-1])
            batch_size = input_ids.shape[0]
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
            batch_size = inputs_embeds.shape[0]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        device = input_ids.device if input_ids is not None else inputs_embeds.device

        if token_type_ids is not None:
            token_type_ids = token_type_ids.view(-1, input_shape[-1])
        if position_ids is not None:
            position_ids = position_ids.view(-1, input_shape[-1])

        if past_key_values is None:
            past_length = 0
            past_key_values = tuple([None] * len(self.h))
        else:
            past_length = past_key_values[0][0].size(-2)
        if position_ids is None:
            position_ids = torch.arange(past_length, input_shape[-1] + past_length, dtype=torch.long, device=device)
            position_ids = position_ids.unsqueeze(0).view(-1, input_shape[-1])

        # GPT2Attention mask.
        if attention_mask is not None:
            if batch_size <= 0:
                raise ValueError("batch_size has to be defined and > 0")
            attention_mask = attention_mask.view(batch_size, -1)
            # We create a 3D attention mask from a 2D tensor mask.
            # Sizes are [batch_size, 1, 1, to_seq_length]
            # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
            # this attention mask is more simple than the triangular masking of causal attention
            # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
            attention_mask = attention_mask[:, None, None, :]

            # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
            # masked positions, this operation will create a tensor which is 0.0 for
            # positions we want to attend and -10000.0 for masked positions.
            # Since we are adding it to the raw scores before the softmax, this is
            # effectively the same as removing these entirely.
            attention_mask = attention_mask.to(dtype=self.dtype)  # fp16 compatibility
            attention_mask = (1.0 - attention_mask) * -10000.0

        # If a 2D ou 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.add_cross_attention and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # head_mask has shape n_layer x batch x n_heads x N x N
        head_mask = self.get_head_mask(head_mask, self.config.n_layer)

        if inputs_embeds is None:
            inputs_embeds = self.wte(input_ids)
        position_embeds = self.wpe(position_ids)
        hidden_states = inputs_embeds + position_embeds

        if token_type_ids is not None:
            token_type_embeds = self.wte(token_type_ids)
            hidden_states = hidden_states + token_type_embeds

        hidden_states = self.drop(hidden_states)

        output_shape = input_shape + (hidden_states.size(-1),)

        presents = () if use_cache else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None
        all_hidden_states = () if output_hidden_states else None
        for i, (block, layer_past) in enumerate(zip(self.h, past_key_values)):

            # Model parallel
            if self.model_parallel:
                torch.cuda.set_device(hidden_states.device)
                # Ensure layer_past is on same device as hidden_states (might not be correct)
                if layer_past is not None:
                    layer_past = tuple(past_state.to(hidden_states.device) for past_state in layer_past)
                # Ensure that attention_mask is always on the same device as hidden_states
                if attention_mask is not None:
                    attention_mask = attention_mask.to(hidden_states.device)
                if isinstance(head_mask, torch.Tensor):
                    head_mask = head_mask.to(hidden_states.device)
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            if self.gradient_checkpointing and self.training:

                if use_cache:
                    logger.warning(
                        "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                    )
                    use_cache = False

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        # None for past_key_value
                        return module(*inputs, use_cache, output_attentions)

                    return custom_forward

                outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(block),
                    hidden_states,
                    None,
                    attention_mask,
                    head_mask[i],
                    encoder_hidden_states,
                    encoder_attention_mask,
                )
            else:
                outputs = block(
                    hidden_states,
                    layer_past=layer_past,
                    attention_mask=attention_mask,
                    head_mask=head_mask[i],
                    encoder_hidden_states=encoder_hidden_states,
                    encoder_attention_mask=encoder_attention_mask,
                    use_cache=use_cache,
                    output_attentions=output_attentions,
                )

            hidden_states = outputs[0]
            if use_cache is True:
                presents = presents + (outputs[1],)

            if output_attentions:
                all_self_attentions = all_self_attentions + (outputs[2 if use_cache else 1],)
                # if self.config.add_cross_attention:
                #     all_cross_attentions = all_cross_attentions + (outputs[3 if use_cache else 2],)

            # Model Parallel: If it's the last layer for that device, put things on the next device
            if self.model_parallel:
                for k, v in self.device_map.items():
                    if i == v[-1] and "cuda:" + str(k) != self.last_device:
                        hidden_states = hidden_states.to("cuda:" + str(k + 1))

        hidden_states = self.ln_f(hidden_states)

        hidden_states = hidden_states.view(*output_shape)
        # Add last hidden state
        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [hidden_states, presents, all_hidden_states, all_self_attentions, all_cross_attentions]
                if v is not None
            )

        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=presents,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )


In [35]:
student_decoder = StudentGPT2Model(student_decoder_config)

In [58]:
sample['decoder_input_ids'].size()

torch.Size([16, 48])

In [59]:
input_embed = student_decoder.wte(sample['decoder_input_ids'])
input_embed.size()

torch.Size([16, 48, 768])

In [60]:
c_attn = StudentLinear(
        "decoder.attn.c_attn", 
        1, 
        4, 
        768, 
        3*768
        )

In [61]:
c_attn(input_embed).size()

torch.Size([16, 48, 2304])

In [62]:
decoder_output = student_decoder(sample['decoder_input_ids'])

In [63]:
len(decoder_output)

4

In [64]:
decoder_output.keys()

odict_keys(['last_hidden_state', 'past_key_values', 'hidden_states', 'attentions'])

In [67]:
decoder_output.hidden_states[0].size()

torch.Size([16, 48, 768])

In [68]:
decoder_output.attentions[0].size()

torch.Size([16, 12, 48, 48])

In [523]:
print(decoder_output[0].size())

torch.Size([16, 62, 768])


In [524]:
len(decoder_output[1])

4

In [69]:
student_decoder

StudentGPT2Model(
  (wte): Embedding(50260, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): StudentGPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): StudentGPT2Attention(
        (c_attn): StudentLinear(
          (weight_generator): WeightGenerator(
            (tanh): Tanh()
          )
          (bias_generator): BiasGenerator(
            (tanh): Tanh()
          )
        )
        (c_proj): StudentLinear(
          (weight_generator): WeightGenerator(
            (tanh): Tanh()
          )
          (bias_generator): BiasGenerator(
            (tanh): Tanh()
          )
        )
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): StudentGPT2MLP(
        (c_fc): StudentLinear(
          (weight_generator): WeightGenerator(
            (tanh

In [526]:
TeacherWeightGroup.teacher_model

GrafomerModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

## Decoder LMHead

In [527]:
print(student_decoder_config.n_embd)
print(student_decoder_config.vocab_size)

768
42000


In [70]:
TeacherWeightGroup.teacher_model.decoder.lm_head

Linear(in_features=768, out_features=50260, bias=False)

In [30]:
class StudentGPT2LMHeadModel(nn.Module, ModuleUtilsMixin, GenerationMixin, PushToHubMixin):
    
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = StudentGPT2Model(config)
        # self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.lm_head = TeacherWeightGroup.teacher_model.decoder.lm_head
        self.init_weights()

        # Model parallel
        self.model_parallel = False
        self.device_map = None
    
    def get_input_embeddings(self):
        return self.transformer.wte

    def get_output_embeddings(self):
        return self.lm_head

    def set_output_embeddings(self, new_embeddings):
        self.lm_head = new_embeddings

    def prepare_inputs_for_generation(self, input_ids, past=None, **kwargs):
        token_type_ids = kwargs.get("token_type_ids", None)
        # only last token for inputs_ids if past is defined in kwargs
        if past:
            input_ids = input_ids[:, -1].unsqueeze(-1)
            if token_type_ids is not None:
                token_type_ids = token_type_ids[:, -1].unsqueeze(-1)

        attention_mask = kwargs.get("attention_mask", None)
        position_ids = kwargs.get("position_ids", None)

        if attention_mask is not None and position_ids is None:
            # create position_ids on the fly for batch generation
            position_ids = attention_mask.long().cumsum(-1) - 1
            position_ids.masked_fill_(attention_mask == 0, 1)
            if past:
                position_ids = position_ids[:, -1].unsqueeze(-1)
        else:
            position_ids = None
        return {
            "input_ids": input_ids,
            "past_key_values": past,
            "use_cache": kwargs.get("use_cache"),
            "position_ids": position_ids,
            # "attention_mask": attention_mask,
            # "token_type_ids": token_type_ids,
        }
    
    def init_weights(self):
        self.apply(self._init_weights)
        self.tie_weights()

    def _init_weights(self, module):
        """Initialize the weights."""
        # if isinstance(module, (nn.Linear, Conv1D)):
        #     # Slightly different from the TF version which uses truncated_normal for initialization
        #     # cf https://github.com/pytorch/pytorch/pull/5617
        #     module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
        #     if module.bias is not None:
        #         module.bias.data.zero_()
        if isinstance(module, nn.Embedding):
            # module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    
    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=True,
        encoder_attention_mask=None,
        labels=None,
        use_cache=None,
        output_attentions=True,
        output_hidden_states=True,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for language modeling. Note that the labels **are shifted** inside the model, i.e. you can set
            ``labels = input_ids`` Indices are selected in ``[-100, 0, ..., config.vocab_size]`` All labels set to
            ``-100`` are ignored (masked), the loss is only computed for labels in ``[0, ..., config.vocab_size]``
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_outputs = self.transformer(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = transformer_outputs[0]

        # Set device for model parallelism
        if self.model_parallel:
            torch.cuda.set_device(self.transformer.first_device)
            hidden_states = hidden_states.to(self.lm_head.weight.device)

        lm_logits = self.lm_head(hidden_states)

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        if not return_dict:
            output = (lm_logits,) + transformer_outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return CausalLMOutputWithCrossAttentions(
            loss=loss,
            logits=lm_logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
            cross_attentions=transformer_outputs.cross_attentions,
        )

    @staticmethod
    def _reorder_cache(past: Tuple[Tuple[torch.Tensor]], beam_idx: torch.Tensor) -> Tuple[Tuple[torch.Tensor]]:
        """
        This function is used to re-order the :obj:`past_key_values` cache if
        :meth:`~transformers.PreTrainedModel.beam_search` or :meth:`~transformers.PreTrainedModel.beam_sample` is
        called. This is required to match :obj:`past_key_values` with the correct beam_idx at every generation step.
        """
        return tuple(
            tuple(past_state.index_select(0, beam_idx.to(past_state.device)) for past_state in layer_past)
            for layer_past in past
        )
    
    def tie_weights(self):
        """
        Tie the weights between the input embeddings and the output embeddings.

        If the :obj:`torchscript` flag is set in the configuration, can't handle parameter sharing so we are cloning
        the weights instead.
        """
        output_embeddings = self.get_output_embeddings()
        if output_embeddings is not None and self.config.tie_word_embeddings:
            self._tie_or_clone_weights(output_embeddings, self.get_input_embeddings())

        if self.config.is_encoder_decoder and self.config.tie_encoder_decoder:
            if hasattr(self, self.base_model_prefix):
                self = getattr(self, self.base_model_prefix)
            self._tie_encoder_decoder_weights(self.encoder, self.decoder, self.base_model_prefix)

        for module in self.modules():
            if hasattr(module, "_tie_weights"):
                module._tie_weights()
    
    def _tie_or_clone_weights(self, output_embeddings, input_embeddings):
        """Tie or clone module weights depending of whether we are using TorchScript or not"""
        if self.config.torchscript:
            output_embeddings.weight = nn.Parameter(input_embeddings.weight.clone())
        else:
            output_embeddings.weight = input_embeddings.weight

        if getattr(output_embeddings, "bias", None) is not None:
            output_embeddings.bias.data = nn.functional.pad(
                output_embeddings.bias.data,
                (
                    0,
                    output_embeddings.weight.shape[0] - output_embeddings.bias.shape[0],
                ),
                "constant",
                0,
            )
        if hasattr(output_embeddings, "out_features") and hasattr(input_embeddings, "num_embeddings"):
            output_embeddings.out_features = input_embeddings.num_embeddings

In [31]:
student_decoder_lmhead_model = StudentGPT2LMHeadModel(student_decoder_config)

In [73]:
output = student_decoder_lmhead_model(sample['decoder_input_ids'])

In [74]:
output.keys()

odict_keys(['logits', 'past_key_values', 'hidden_states', 'attentions'])

In [533]:
len(output.hidden_states)

5

In [534]:
output[0].size()

torch.Size([16, 62, 50260])

In [535]:
len(output[1])

4

# Student Graformer

In [630]:
TeacherWeightGroup.teacher_model

GrafomerModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [631]:
student_encoder

StudentBertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): StudentBertEncoder(
    (layer): ModuleList(
      (0): StudentBertLayer(
        (attention): StudentBertAttention(
          (self): StudentBertSelfAttention(
            (query): StudentLinear(
              (weight_generator): WeightGenerator(
                (tanh): Tanh()
              )
              (bias_generator): BiasGenerator(
                (tanh): Tanh()
              )
            )
            (key): StudentLinear(
              (weight_generator): WeightGenerator(
                (tanh): Tanh()
              )
              (bias_generator): BiasGenerator(
                (tanh): Tanh()
              )
            )
            (value)

In [632]:
student_decoder_lmhead_model

StudentGPT2LMHeadModel(
  (transformer): StudentGPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): StudentGPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): StudentGPT2Attention(
          (c_attn): StudentLinear(
            (weight_generator): WeightGenerator(
              (tanh): Tanh()
            )
            (bias_generator): BiasGenerator(
              (tanh): Tanh()
            )
          )
          (c_proj): StudentLinear(
            (weight_generator): WeightGenerator(
              (tanh): Tanh()
            )
            (bias_generator): BiasGenerator(
              (tanh): Tanh()
            )
          )
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): StudentGPT2MLP

In [32]:
from model import GraftAttentionModule

class StudentGrafomerModel(nn.Module, ModuleUtilsMixin, GenerationMixin, PushToHubMixin):
    def __init__(self, enc_name, dec_name):
        super().__init__()

        self.encoder = enc_name
        self.decoder = dec_name

        self.config = self.decoder.config  # for compatibility in generate method
        self.config.is_encoder_decoder = True
        self.config.decoder_start_token_id = 2
        print(self.config)

        self.decoder_body = getattr(self.decoder, 'transformer')
        self.decoder_head = getattr(self.decoder, 'lm_head')

        self.bart_config = AutoConfig.from_pretrained("facebook/bart-base")
        
        self.decoder_embed_dim = 768
        self.graft_module_config = {'num_enc_layer': 2, 'num_dec_layer': 2}
        self.graft_module = GraftAttentionModule(self.bart_config, self.graft_module_config, self.decoder_embed_dim)
    
    def get_encoder(self):
        return self.encoder
        
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        decoder_input_ids=None,
        decoder_attention_mask=None,
        head_mask=None,
        decoder_head_mask=None,
        cross_attn_head_mask=None,
        encoder_outputs=None,
        past_key_values=None,
        inputs_embeds=None,
        decoder_inputs_embeds=None,
        use_cache=None,
        output_attentions=True,
        output_hidden_states=True,
        return_dict=None,
    ):

        if encoder_outputs is None:
            encoder_outputs = self.encoder(input_ids=input_ids,
                                        attention_mask=attention_mask,
                                        head_mask=head_mask,
                                        inputs_embeds=inputs_embeds,
                                        output_attentions=output_attentions,
                                        output_hidden_states=output_hidden_states,
                                        return_dict=return_dict)
        
        # If the user passed a tuple for encoder_outputs, we wrap it in a BaseModelOutput when return_dict=True
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(last_hidden_state=encoder_outputs[0],
                                            hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                                            attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None)

        # train
        if decoder_input_ids is not None:
            decoder_outputs = self.decoder_body(input_ids=decoder_input_ids,
                                                attention_mask=decoder_attention_mask,
                                                use_cache=use_cache)

            mask = _expand_mask(attention_mask, self.dtype)
            dec_mask = _make_causal_mask(decoder_attention_mask.shape, self.dtype).to(self.device) + _expand_mask(decoder_attention_mask, self.dtype)
            cross_mask = _expand_mask(attention_mask, self.dtype, tgt_len=decoder_input_ids.shape[1])
        
        # eval
        else:
            decoder_outputs = self.decoder_body(input_ids=input_ids, use_cache=use_cache)

            bsz, sql = input_ids.shape
            mask = _expand_mask(attention_mask, self.dtype)
            dec_mask = _make_causal_mask([bsz, sql], self.dtype).to(self.device)
            cross_mask = _expand_mask(attention_mask, self.dtype, tgt_len=sql)


        # Attention
        # Hidden layer Output
        # Prediction Output
        encoder_hidden_state = encoder_outputs[0]
        decoder_hidden_state = decoder_outputs[0]
        encoder_hidden_states = encoder_outputs.hidden_states
        decoder_hidden_states = decoder_outputs.hidden_states
        encoder_attentions = encoder_outputs.attentions
        decoder_attentions = decoder_outputs.attentions

        graformer_hidden_state = self.graft_module(
            encoder_hidden_states=encoder_hidden_state,
            encoder_attention_mask=mask,
            decoder_hidden_states=decoder_hidden_state,
            decoder_attention_mask=dec_mask,
            cross_attention_mask=cross_mask,
            output_attentions=output_attentions,
        )

        # print
        output_hidden_states = decoder_hidden_state + graformer_hidden_state
        output_hidden_states = self.decoder_head(output_hidden_states)

        return Seq2SeqLMOutput(
            logits=output_hidden_states,
            encoder_last_hidden_state=encoder_hidden_state,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attentions=encoder_attentions,
            # decoder_last_hidden_state=decoder_hidden_state,
            decoder_hidden_states=decoder_hidden_states,
            decoder_attentions=decoder_attentions,
            # graft_hidden_states=graformer_hidden_state,
            # graft_attentions=graft_attentions,
        )
        
    def prepare_inputs_for_generation(self, input_ids, past=None, **kwargs):
        if past is not None:
            # print(past, type(past))
            # print(len(past))
            # print(past.shape)
            pass
        return {"input_ids": input_ids, **kwargs}


In [33]:
student_grafomer = StudentGrafomerModel(student_encoder, student_decoder_lmhead_model)

StudentDecoderConfig {
  "_name_or_path": [
    "kykim/gpt3-kor-small_based_on_gpt2"
  ],
  "activation_function": "gelu_new",
  "architectures": [
    [
      "GPT2Model"
    ]
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 3,
  "decoder_start_token_id": 2,
  "embd_pdrop": 0.1,
  "eos_token_id": 3,
  "gradient_checkpointing": [
    false
  ],
  "initializer_range": 0.02,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 4,
  "n_positions": 2048,
  "reorder_and_upcast_attn": [
    false
  ],
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": [
    false
  ],
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 42000
}



In [34]:
student_grafomer.to("cuda:0")

StudentGrafomerModel(
  (encoder): StudentBertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): StudentBertEncoder(
      (layer): ModuleList(
        (0): StudentBertLayer(
          (attention): StudentBertAttention(
            (self): StudentBertSelfAttention(
              (query): StudentLinear(
                (weight_generator): WeightGenerator(
                  (tanh): Tanh()
                )
                (bias_generator): BiasGenerator(
                  (tanh): Tanh()
                )
              )
              (key): StudentLinear(
                (weight_generator): WeightGenerator(
                  (tanh): Tanh()
                )
                (bias_generator): BiasGenera

In [35]:
student_grafomer_output = student_grafomer(sample["input_ids"], sample["attention_mask"], sample["decoder_input_ids"], sample["decoder_attention_mask"])

In [ ]:
student_grafomer_output

In [36]:
student_grafomer_output.keys()

odict_keys(['logits', 'decoder_hidden_states', 'decoder_attentions', 'encoder_last_hidden_state', 'encoder_hidden_states', 'encoder_attentions'])

In [37]:
teacher_output = model(sample["input_ids"], sample["attention_mask"], sample["decoder_input_ids"], sample["decoder_attention_mask"])

In [38]:
teacher_output.keys()

odict_keys(['logits'])

# Loss test

In [94]:
from collections import deque


class SelectiveDistilationLoss(nn.Module):

    queue = deque()

    def __init__(self, pad_token_id):
        super().__init__()
        self.loss_ce_function = nn.NLLLoss(reduction='none', ignore_index=pad_token_id)
        self.loss_kd_function = nn.KLDivLoss(reduction='none')

    def forward(self, teacher_logits, student_logits, labels, top_r, alpha):

        loss = 0

        loss_ce = self.loss_ce_function(student_logits, labels)
        loss_kd = self.loss_kd_function(teacher_logits, student_logits)
        
        SelectiveDistilationLoss.queue.extend(torch.tensor(list(zip(labels, loss_ce))))

        while len(SelectiveDistilationLoss.queue) > 50000:
            SelectiveDistilationLoss.queue.popleft()
        
        hard_threshold = int(len(SelectiveDistilationLoss.queue) * top_r) # 0 < top_r <= 1.0
        sorted_queue = sorted(SelectiveDistilationLoss.queue, key=lambda x: x[1], reverse=True)
        hard_words = set(map(lambda x: x[0], sorted_queue[:hard_threshold]))

        for label, loss_ce, loss_kd in zip(labels, loss_ce, loss_kd):
            if label in hard_words:
                print("hard")
                loss += loss_ce + alpha * loss_kd
            else:
                print("not hard")
                loss += loss_ce
        
        return loss


class AllDistillationLoss(nn.Module):

    def __init__(self, pad_token_id):
        super().__init__()
        self.loss_ce_function = nn.CrossEntropyLoss(ignore_index=pad_token_id)
        self.loss_kd_function = nn.KLDivLoss(reduction='batchmean')
    
    def forward(self, teacher_logits, student_logits, labels, alpha):

        loss_ce = self.loss_ce_function(student_logits, labels)
        loss_kd = self.loss_kd_function(teacher_logits, student_logits)

        return loss_ce + (alpha * loss_kd)

In [95]:
loss_function = AllDistillationLoss(pad_token_id=3)

student_logits = student_grafomer_output.logits.view(-1, len(decoder_tokenizer))
teacher_logits = teacher_output.logits.view(-1, len(decoder_tokenizer))
labels = sample["labels"].view(-1)

loss = loss_function(teacher_logits, student_logits, labels, 0.5)

In [96]:
loss

tensor(2197.3611, device='cuda:0', grad_fn=<AddBackward0>)

In [63]:
loss_function = SelectiveDistilationLoss(pad_token_id=3)

student_logits = student_grafomer_output.logits.view(-1, len(decoder_tokenizer))
teacher_logits = teacher_output.logits.view(-1, len(decoder_tokenizer))
labels = sample["labels"].view(-1)

loss = loss_function(teacher_logits, student_logits, labels, 0.7, 0.5)

not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
not hard
n

In [61]:
loss

tensor(-19.6120, device='cuda:0', grad_fn=<AddBackward0>)

In [658]:
student_grafomer_output.encoder_attentions[0].size()

torch.Size([16, 12, 125, 125])

In [553]:
student_grafomer_output[0].size()

torch.Size([16, 62, 50260])

In [544]:
student_grafomer_output.logits.size()

torch.Size([16, 62, 50260])

In [545]:
student_grafomer_output.keys()

odict_keys(['logits'])

In [314]:
student_grafomer

StudentGrafomerModel(
  (encoder): StudentBertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): StudentBertEncoder(
      (layer): ModuleList(
        (0): StudentBertLayer(
          (attention): StudentBertAttention(
            (self): StudentBertSelfAttention(
              (query): StudentLinear(
                (weight_generator): WeightGenerator(
                  (tanh): Tanh()
                )
                (bias_generator): BiasGenerator(
                  (tanh): Tanh()
                )
              )
              (key): StudentLinear(
                (weight_generator): WeightGenerator(
                  (tanh): Tanh()
                )
                (bias_generator): BiasGenera